In [2]:
!pip install python-telegram-bot --upgrade
!pip install dialogflow
!pip install pymorphy2
!pip install stop_words
!pip install annoy

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (13.0)


In [3]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import dialogflow
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [4]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

In [5]:
updater = Updater("1397773161:AAHJo-f4XLKYf0Lr05jP6guLoa7QH63ZxZ8", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/telegramagent-dmst-3fc2f762152f.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'telegramagent-dmst' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'dimasikbot'  # ID бота из телеграма

In [6]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/'My Drive'/Geekbrains.zip /content/
!unzip /content/Geekbrains.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/Geekbrains.zip
replace Geekbrains/Otvety.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
sentences = []
c = 0
with open("Geekbrains/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)

2020-11-15 11:09:41,367 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts_ru/data
2020-11-15 11:09:41,422 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


2020-11-15 11:17:48,743 - gensim.models.word2vec - INFO - collecting all words and their counts
2020-11-15 11:17:48,744 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-15 11:17:48,811 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 173060 words, keeping 41540 word types
2020-11-15 11:17:48,885 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 353487 words, keeping 68403 word types
2020-11-15 11:17:48,963 - gensim.models.word2vec - INFO - PROGRESS: at sentence #30000, processed 527016 words, keeping 91256 word types
2020-11-15 11:17:49,042 - gensim.models.word2vec - INFO - PROGRESS: at sentence #40000, processed 702279 words, keeping 113049 word types
2020-11-15 11:17:49,121 - gensim.models.word2vec - INFO - PROGRESS: at sentence #50000, processed 885642 words, keeping 134773 word types
2020-11-15 11:17:49,208 - gensim.models.word2vec - INFO - PROGRESS: at sentence #60000, proce

In [9]:
shop_data = pd.read_csv("ProductsDataset.csv")

In [10]:
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

2020-11-15 11:20:00,323 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [11]:
texts = [" ".join(val) for val in shop_data['text'].values]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(texts)

In [13]:
np.mean(tfidf_vect.idf_)

10.39264055035947

In [14]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
#idfs

In [15]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [16]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [17]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([758], [1.4142135381698608])

In [ ]:
updater = Updater(token='1397773161:AAHJo-f4XLKYf0Lr05jP6guLoa7QH63ZxZ8') # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

def textMessage(update: Update, context: CallbackContext):
  input_txt = preprocess_txt(update.message.text)
  vect_ft = embed_txt(input_txt, idfs, midf)
  ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
  for item in ft_index_shop_val:
    title, image = index_map_shop[item]
    update.message.reply_text(title)
    update.message.reply_text(image)
  return
   
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

2020-11-15 12:02:21,461 - apscheduler.scheduler - INFO - Scheduler started
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
